<a href="https://colab.research.google.com/github/benzine01/Ebenezer_Ise/blob/master/Code_Stage_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Importing the dataset
import pandas as pd
 import numpy as np
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')
#print to check for distribution
#of target variable
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [5]:
#check the distribution of target variables
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [6]:
#Note that there are differences in target variables and the classes
#Check for null values in the dataset
df.isnull().sum()
#conversly 
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

**NOTE**
From the dataset, 20472 rows have missing values or null values, this can affect our analysis


In [7]:
#For easy resolution, we will drop the rows with missing values
df.dropna(inplace= True)
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

**NOTE:**
An obvious change in our target variable after removing 
the missing values is that there
are only three classes left #and from the distribution of the 3 classes, we can see that
there is an obvious imbalance between the classes. #There are methods that can be applied to
handle this imbalance such as oversampling and undersampling.
Oversampling involves increasing the number of instances in the class with fewer instances
while undersampling #involves reducing the data points in the class with more instances.
For now, we will convert this to a binary classification problem by combining class '2A'
and '1A'.


---



In [8]:
#reset the dataframe index, to corrrect the 
#effect of the null values dropped from the dataset
df = df.reset_index(drop= True)
#now, we'll convert this to a binary classification problem 
#by combining class '2A' and '1A'.
df['QScore'] = df['QScore'].replace(['1A'], '2A')
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [9]:
#Seperating the target variable and
#undesampling the 3A class
df_2A = df[df.QScore=='2A']
df_3A = df[df.QScore=='3A'].sample(350)
data_df = df_2A.append(df_3A).reset_index(drop=True)
data_df

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Algeria,2016,4,AreaPerCap,2.072989e-01,8.112722e-01,0.048357265,2.258528e-02,2.998367e-02,0.000000,1.119497e+00,2A
1,Algeria,2016,4,AreaTotHA,8.417600e+06,3.294260e+07,1963600,9.171000e+05,1.217520e+06,0.000000,4.545842e+07,2A
2,Algeria,2016,4,BiocapPerCap,2.021916e-01,2.636077e-01,0.027166736,7.947991e-03,2.924496e-02,0.000000,5.301590e-01,2A
3,Algeria,2016,4,BiocapTotGHA,8.210214e+06,1.070408e+07,1103135.245,3.227369e+05,1.187524e+06,0.000000,2.152769e+07,2A
4,Algeria,2016,4,EFConsPerCap,6.280528e-01,1.810332e-01,0.162800822,1.472910e-02,2.924496e-02,1.391455,2.407316e+00,2A
...,...,...,...,...,...,...,...,...,...,...,...,...
585,Lesotho,1977,122,EFProdTotGHA,3.154166e+05,8.036874e+05,628609.4006,0.000000e+00,3.429497e+04,440445.659400,2.222454e+06,3A
586,Argentina,1990,9,BiocapTotGHA,4.840903e+07,7.080900e+07,33514312.82,6.966262e+07,2.197896e+06,0.000000,2.245929e+08,3A
587,Congo,1963,46,AreaPerCap,4.831822e-01,8.914801e+00,20.25774171,7.442968e-01,3.255097e-02,0.000000,3.043257e+01,3A
588,Pakistan,2007,165,BiocapPerCap,3.370932e-01,4.354961e-03,0.012322439,4.009511e-02,4.061064e-02,0.000000,4.344764e-01,3A


In [11]:
#using the scikit-learn utils function to shuffle the target variable
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df.QScore.value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [12]:
#let's view the datatypes of the dataframe after reseting
data_df.dtypes

country            object
year                int64
country_code        int64
record             object
crop_land         float64
grazing_land      float64
forest_land        object
fishing_ground    float64
built_up_land     float64
carbon            float64
total             float64
QScore             object
dtype: object

**Note:* 
We will drop country code, country and year bcause they are too ambigious for our prediction

In [13]:
#encoding the categorical feature
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data_df.record = encoder.fit_transform(data_df.record)
data_df.record

0      6
1      3
2      3
3      5
4      6
      ..
585    1
586    5
587    7
588    7
589    6
Name: record, Length: 590, dtype: int64

In [ ]:
#preprocessing and unfitting columns dropping
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
X = data_df.drop(columns='QScore')
y = data_df['QScore']

In [22]:
#now, let's split the data into training and testing sets
from sklearn.model_selection import train_test_split
X = data_df.drop(columns='QScore')
y = data_df['QScore']
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y_train.value_counts()

3A    247
2A    166
Name: QScore, dtype: int64

In [47]:
#we install/import imblear
import imblearn
import numpy as np
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
X_train_balanced, y_balanced = smote.fit_sample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#using minmaxscaler to scalr every other feature aside the record column
#beacuse the record column is not a numerical feature but a categorical one
from sklearn.preprocessing import  MinMaxScaler
scaler = MinMaxScaler()
#Scaling the x_train set without the record feature
normalized_train_df = scaler.fit_transform(X_train_balanced.(columns=['record']))

#passing the scaled data into the dataframe
normalized_train_df = pd.DaraFrame(normalized_train_df, columns=X-X_train_balanced.drop(columns=['record']).columns)

#adding the record colums back
normalized_train_df['record'] = X_train_balanced['record']

In [ ]:
#repeat for x test
#reset the index for easy computation
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

normalized_test_df = scaler.transform(x_test.drop(columns=['record']))
normalized_test_df = pd.DataFrame(normalized_test_df, columns=X_test.drop(columns=['record']).columns)
normalized_test_df['record'] = x_test['record']

The cell above prepares the data for machine learning

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalized_train_df, y_balanced)
new_predictions = log_reg.predict(normalized_train_df)
new_predictions

In [ ]:
#Runnning logisticregression.__dict__
LogisticRegression().__dict__
